# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Dans le carnet précédent, nous avons effectué une TWAS du diabète de type 2 pour 7 tissus GTEx en utilisant les modèles de prédiction de l'expression génique de PredictDB et le modèle TWAS S-PrediXcan. Ici, nous explorerons les résultats pour mieux comprendre la TWAS. Notre première tâche consiste à résumer le nombre de gènes significatifs par tissu. Calculons d'abord les seuils de valeur P

| Tissu                | N gènes testés | Bonferroni global | Bonferroni par tissu |
| :------------------: | -------------- | ----------------- | -------------------- |
| Tissu adipeux sous-cutané |            |                   |                      |
| ...                  |                |                   |                      |

In [ ]:
library(ggplot2)

In [ ]:
tissues <- c(
    "en_Adipose_Subcutaneous",
    "en_Artery_Coronary",
    "en_Brain_Cortex",
    "en_Liver",
    "en_Muscle_Skeletal",
    "en_Pancreas",
    "en_Whole_Blood"
)

df <- data.frame()
for (tissue in tissues) {
    cur <- read.csv(paste0("/workshop/local/results/twas_", tissue, ".csv"))
    df <- rbind(df, data.frame(tissue = tissue, n_tested_genes = nrow(cur)))
}

df$global_bonf <- 0.05 / sum(df$n_tested_genes)
df$tissue_bonf <- 0.05 / df$n_tested_genes
df

Nous pouvons constater que même une correction de Bonferroni complète tenant compte de tous les tests à travers les tissus serait moins stricte que le seuil de signification génome-entier ($5 \times 10^{-8}$). Le nombre de tissus testés varie entre 3 765 pour le foie et 8 626 pour le tissu adipeux sous-cutané, ce qui explique pourquoi les seuils de Bonferroni spécifiques aux tissus peuvent varier. Nous pouvons examiner le nombre de gènes significatifs selon l'approche utilisée. Nous ajouterons également la correction du taux de fausses découvertes de Benjamini Hochberg.

<div class="alert alert-info">
Pourquoi le nombre de gènes testés varie-t-il selon les tissus ?
</div>

In [ ]:
n_sig <- data.frame()
sig_genes_data <- data.frame()
sig_genes <- list()

for (tissue in tissues) {
    cur <- read.csv(paste0("/workshop/local/results/twas_", tissue, ".csv"))

    sig_global_bonf <- cur$pvalue <= 1.174205e-6
    sig_tissue_bonf <- cur$pvalue <= 0.05 / nrow(cur)
    q <- p.adjust(cur$pvalue, method = "BH")
    sig_fdr <- q <= 0.05
    
    sig_genes_data_cur <- cur[sig_fdr, c("gene_name", "pvalue", "effect_size", "n_snps_in_model", "n_snps_used", "pred_perf_pval")]
    sig_genes_data_cur$tissue <- tissue
    sig_genes_data <- rbind(sig_genes_data, sig_genes_data_cur)
    
    sig_genes[[tissue]] <- list(
        sig_global_bonf = cur[sig_global_bonf, "gene_name"],
        sig_tissue_bonf = cur[sig_tissue_bonf, "gene_name"],
        sig_fdr = cur[sig_fdr, "gene_name"]
    )
    
    n_sig <- rbind(n_sig, data.frame(
        tissue = tissue,
        sig_global_bonf = sum(sig_global_bonf),
        sig_tissue_bonf = sum(sig_tissue_bonf),
        sig_fdr = sum(sig_fdr)
    ))
}
n_sig

<div class="alert alert-info">
    Quelle correction est la plus stricte ? La moins stricte ? Quelle approche est la plus appropriée et quelles sont les hypothèses des différentes méthodes ?
</div>

In [ ]:
cat("Gènes significatifs à FDR <= 5% à travers les tissus:\n\n")
for (tissue in tissues) {
    cat(paste0(
        "\t'", tissue, "': ",
        paste0(sig_genes[[tissue]]$sig_fdr, collapse = ", "),
        "\n"
    ))
}

<div class="alert alert-info">
    <p>L'expression génétiquement prédite d'<em>AP3S2</em>, dans tous les tissus considérés, est associée au diabète de type 2. Pour d'autres gènes, les effets ne sont observés que dans certains tissus.</p>
    <p>Par exemple, l'expression de FTO n'est associée au diabète de type 2 que dans le muscle squelettique. Qu'est-ce qui pourrait expliquer cette observation ?</p>

<!-- Sujets de discussion / réponses :
- L'effet est spécifique au tissu.
- L'expression du gène n'est héritable que dans le muscle squelettique.
- Il y avait plus de puissance pour détecter les eQTL ou estimer leur effet dans ce tissu.
//-->

</div>

Nous pouvons visualiser les résultats TWAS à l'aide d'un diagramme en points pour voir plus d'informations sur les associations TWAS, y compris sur la directionnalité des effets.

In [ ]:
source("/workshop/utilities/gene_tissue_dotplot.R")
options(repr.plot.width = 10, repr.plot.height = 5)
options(repr.plot.res = 200)
plot_gene_tissue_dots_advanced(sig_genes_data)

<div class="alert alert-info">
Si nous supposons que l'association est causale, quel gène devrions-nous inhiber pour prévenir le diabète de type 2 ? Dans quel(s) tissu(s) ?
    
<!-- Réponse / sujets
WFS1 dans le tissu adipeux ou ANK1 dans les muscles pourraient être de bons choix. Nous voulons des gènes qui sont positivement et robustement associés au diabète de type 2 (rouge).
//-->
</div>

Nous extrairons les statistiques sommaires GWAS pour le gène _WFS1_ et montrerons le graphique de région tout en soulignant les variants inclus dans le modèle de prédiction.

In [ ]:
# Nous définissons la région WFS1 en utilisant les données d'Ensembl.
# Voir : https://useast.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000109501;r=4:6269849-6303265
# Nous avons ajouté un espacement de +/- 100kb.
wfs1_region <- list(
    chrom = "chr4",
    start = 6269849 - 100000,
    end = 6303265 + 100000
)

get_predictdb_and_gwas_data <- function(region, gene_name, tissue) {
    # Nous lisons les statistiques sommaires GWAS, mais gardons seulement l'information pour
    # les variants dans la région d'intérêt.
    gwas <- read.csv(
        paste0(
            "/workshop/data/PrediXcan/gwas_harmonized/harmonized_",
            tissue,
            ".tsv.gz"
        ), sep="\t"
    )
    gwas_region <- gwas[
        (gwas$chromosome == region$chrom) &
        (region$start <= gwas$base_pair_location) &
        (gwas$base_pair_location <= region$end),
    ]
    rm(gwas)

    # Nous extrairons le rsid et les poids des variants inclus dans le
    # modèle prédictif de l'expression génique.
    db_filename = paste0("/workshop/data/PrediXcan/predictdb/", tissue, ".db")
        
    model <- system(paste0(
        "sqlite3 ",
        db_filename,
        " \"select rsid, weight from weights w, extra e where e.gene=w.gene and e.genename='",
        gene_name, "';\""    ), intern=T)

    if (length(model) == 0) {
        stop("Impossible de trouver les données PredictDB pour le gène.")
    }

    model <- data.frame(do.call(rbind, strsplit(model, "\\|")))
    colnames(model) <- c("rsid", "weight")
    model$weight <- as.numeric(model$weight)

    # Nous créons des variables indicatrices pour tester si un variant est dans le modèle
    # et nous définissons une valeur par défaut pour les poids à des fins de visualisation.
    gwas_region$in_model <- gwas_region$rsid_x %in% model$rsid

    gwas_region <- merge(gwas_region, model, by.x = "rsid_x", by.y = "rsid", all.x = T)
    min_abs_weight = min(abs(na.omit(gwas_region$weight)))
    gwas_region$weight <- ifelse(
        is.na(gwas_region$weight),
        0.8 * min_abs_weight,
        gwas_region$weight
    )
    
    list(
        model = model,
        gwas_region = gwas_region
    )
}

plot_region <- function(region, gene_name, tissue) {
    data <- get_predictdb_and_gwas_data(region, gene_name, tissue)
    
    options(repr.plot.width = 10, repr.plot.height = 4)
    options(repr.plot.res = 200)
    
    plot <- ggplot(data$gwas_region, aes(x = base_pair_location, y = neg_log_10_p_value,
                      color = in_model, size = abs(weight))) +
        geom_point(alpha = 0.7) +
        scale_color_manual(values = c("FALSE" = "#222222", "TRUE" = "#2E86C1")) +
        labs(x = paste0("Position des paires de bases (", region$chr, " ,GRCh38)"),
             y = "-log10(P)",
             color = paste0("Dans le modèle ", gene_name)) +
        theme_minimal()
    
    plot
}

In [ ]:
plot_region(wfs1_region, "WFS1", "en_Adipose_Subcutaneous")

<div class="alert alert-info">
    <p>
    Le graphique ci-dessus montre les statistiques d'association des variants génétiques dans la région <em>WFS1</em> avec le diabète de type 2. Les points bleus représentent les variants qui ont été inclus dans le modèle prédisant l'expression de <em>WFS1</em> dans le tissu adipeux, la taille du point étant proportionnelle au poids.
    </p>
    <p>
    Ce graphique appuie-t-il les résultats TWAS ? Pouvez-vous imaginer comment ce graphique pourrait informer l'interprétation des résultats ?
    </p>
    <p>
    Expérimentez avec les appels suivants pour visualiser les signaux d'association dans d'autres tissus. Qu'observez-vous ? Vous pouvez également essayer la fonction avec d'autres gènes !
    </p>
</div>

In [ ]:
# plot_region(wfs1_region, "WFS1", "en_Whole_Blood")
# plot_region(wfs1_region, "WFS1", "en_Pancreas")
# plot_region(wfs1_region, "WFS1", "en_Muscle_Skeletal")